In [1]:
from pynq import Overlay, GPIO, Register, allocate, MMIO
import os
from convert import *
import numpy as np

import struct


In [2]:
converter=Converter()


In [3]:
overlay = Overlay("fwd_fcc_test.bit")
fwip=overlay.forward_fcc_0
fwip.register_map



RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED=0),
  x = Register(x=0),
  w = Register(w=0),
  y = Register(y=0),
  b = Register(b=0),
  xdim = Register(xdim=0),
  ydim = Register(ydim=0)
}

In [4]:

dim =3
xbuff= allocate(shape=(dim,), dtype='uint16')
wbuff= allocate(shape=(dim,), dtype='uint16')
ybuff= allocate(shape=(1,), dtype='uint16')
bbuff= allocate(shape=(1,), dtype='uint16')






In [5]:
dim=3

BASE_ADDR=0x0
BASE_ADDRESS_X=xbuff.device_address
BASE_ADDRESS_W=wbuff.device_address
BASE_ADDRESS_B=bbuff.device_address
BASE_ADDRESS_Y=ybuff.device_address


mmio_x=MMIO(BASE_ADDRESS_X,dim*4)
mmio_w=MMIO(BASE_ADDRESS_W,dim*4)
mmio_b=MMIO(BASE_ADDRESS_B,4)
mmio_y=MMIO(BASE_ADDRESS_Y,4)



# mmio_x.write(0,int(converter.encode(0.1)))
# mmio_x.write(4,int(converter.encode(0.1)))
# mmio_x.write(8,int(converter.encode(0.1)))

# mmio_w.write(0,int(converter.encode(2.0)))
# mmio_w.write(4,int(converter.encode(3.0)))
# mmio_w.write(8,int(converter.encode(4.0)))

# mmio_b.write(0,int(converter.encode(0.0)))

# mmio_y.write(0,int(converter.encode(0)))
x=[0.1,0.1,0.1]
xbuff[:]= [converter.encode(x[i]) for i in range(dim)]

w=[2,3,4]
wbuff[:]= [converter.encode(w[i]) for i in range(dim)]

bbuff[:]=[converter.encode(1.0)]

ybuff[:]=[converter.encode(0)]

xbuff.flush()
wbuff.flush()
bbuff.flush()
ybuff.flush()

In [6]:
fwip.register_map.x=BASE_ADDRESS_X
fwip.register_map.w=BASE_ADDRESS_W
fwip.register_map.y=BASE_ADDRESS_Y
fwip.register_map.b=BASE_ADDRESS_B
fwip.register_map.xdim=dim
fwip.register_map.ydim=1

In [7]:
fwip.write(0x00, 1)
fpga_state = fwip.read(0x00)

max_try = 1000000
while fpga_state != 6 and fpga_state != 4:
    fpga_state = fwip.read(0x00)
    max_try = max_try -1
    if max_try == 0:
        print("ERROR: Can't go ahead")
        fwip.write(0x00, 4)
        break
fwip.write(0x00, 4)

In [8]:
# val=converter.decode(ybuff[0])
# print(val)

ybuff.invalidate()

# print(converter.decode(ybuff[:]))
print(xbuff)
print(wbuff)
print(bbuff)
print(ybuff)

[12 12 12]
[256 384 512]
[128]
[236]


In [12]:
fwip.register_map


RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED=0),
  x = Register(x=0),
  w = Register(w=16),
  y = Register(y=32),
  b = Register(b=24),
  xdim = Register(xdim=3),
  ydim = Register(ydim=1)
}